In [22]:
import json
import re
import core.utils as oa
from rapidfuzz import fuzz
import pandas as pd
import io
import os

import datetime
from numpyencoder import NumpyEncoder
from pathlib import Path

# root directory path
ROOT = Path(os.getcwd()).resolve().parents[0]

In [23]:
date = datetime.datetime.now().strftime("%y-%m-%d_%H:%M")
date

'25-09-20_12:59'

In [3]:
print("Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "1":
    task = "bibel"
    print("Bibelstellen werden gesucht")
else:
    task = "lieder"
    print("Liedzitate werden gesucht")


Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?
Liedzitate werden gesucht


In [4]:
print("Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "2":
    corpus = "longest"
else:
    corpus = "most"


with open(ROOT / f"sermons_with_{corpus}_music.json", "r") as f:
    testsermons = json.load(f)

Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?


In [ ]:
relevant_page_texts = []
if task == "lieder":
    for n in range(41, 1291):
        with open(ROOT / f"source_texts/praxis_pietatis_verses/{n}.json") as f:
            page = json.load(f)
        page_info = {}
        page_info[n] = page
        relevant_page_texts.append(page_info)
else:
    with open(ROOT / "source_texts/bible/old_testament_chunked.json") as f:
        old_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)
    with open(ROOT / f"source_texts/bible/new_testament_chunked.json") as f:
        new_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)

In [6]:
fuzziness = 80

In [7]:
similarity_table = {}
similarity_table["task"] = task
similarity_table['date'] = date
similarity_table['corpus'] = corpus
similarity_table['method'] = 'similarity_search'
similarity_table['fuzziness'] = fuzziness


In [8]:
testsermons = ["E000036"]

In [14]:
import core.similaritysearch as simsearch

similarity_table['results'] = simsearch.find_similarities("lieder", testsermons, relevant_page_texts, 80)


Starting with E000036


In [36]:
guessed_hits = remove_duplicates(guessed_hits).reset_index(drop=True)

Psalm_73_26
Psalm_73_25
Psalm_73_26
Psalm_10_11
Jeremia_42_9
Psalm_38_9
Psalm_94_19
Psalm_116_7
Jesaja_52_7
Psalm_37_4
Jesaja_52_7
Jesaja_52_7
Psalm_64_10
Jesaja_52_7
Psalm_64_10
Psalm_100_4
Psalm_100_4
3 Mose_14_48
Psalm_100_4
3 Mose_14_48
Psalm_118_15
2 Könige_13_21
Daniel_3_14
Jeremia_11_17
Sprüche_9_7
Hesekiel_34_16
Nehemia_8_10
4 Mose_15_24
2 Könige_1_6
Hesekiel_34_16
Rut_3_12
Jesaja_30_32
Psalm_33_2
Psalm_33_3
2 Mose_26_20
Klagelieder_3_26
Hesekiel_41_20
Psalm_87_7
Hesekiel_33_33
5 Mose_18_6
Psalm_4_5
Daniel_2_20
Psalm_81_3
1 Chronik_29_17
Amos_5_23
1 Chronik_29_17
Psalm_137_1
Hohelied_6_3
Hohelied_7_11
Psalm_71_12
Jeremia_30_11
Psalm_126_3
Psalm_73_23
Psalm_73_24
Psalm_16_11
Hohelied_4_16
Psalm_16_11
2 Könige_5_16
Psalm_30_6
Psalm_132_13
Psalm_111_2


In [37]:
guessed_hits

,Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung
0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True
1,wenn mir gleich leib und seel verschmacht,21,16,Psalm_73_26,Wenn mir gleich Leib vnd Seele verschmacht,89.16,True
2,der wird nimmermehr sterben,23,8,Psalm_10_11,Er wirds nimer mehr sehen,80.77,False
3,der mich gesandt hat,23,15,2 Samuel_24_13,dem der mich gesand hat,88.37,True
4,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.00,True
...,...,...,...,...,...,...,...
56,für dem ich stehe,159,17,2 Könige_5_16,fur dem ich stehe,94.12,True
57,sondern hast lust zum leben,163,4,Psalm_30_6,Vnd er hat lust zum Leben,84.62,True
58,und hast lust in demselben zu wohnen,163,12,Psalm_132_13,Vnd hat lust daselbs zu wonen,80.00,True
59,wer ihr achtet der hat eitel lust daran,163,31,Psalm_111_2,Wer jr achtet der hat eitel lust dran,92.11,True


In [38]:
guessed_hits.to_csv("../predictions/E000036_simsearch_bibel_80.csv")

In [36]:
similarity_table

{'task': 'bibel',
 'date': '25-09-19_12:38',
 'corpus': 'most',
 'method': 'similarity_search',
 'fuzziness': 90,
 'results': [['E000036',
   1733,
   'Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung\n0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,False\n1,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True\n2,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.0,False\n3,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.0,True\n4,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,False\n5,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,True\n6,der wird dir geben was dein hertz wünschet,23,47,Psalm_37_4,Der wird dir geben was dein hertz wündschet,96.47,False\n7,der wird dir geben was dein hertz wünschet,23,47,Psal

In [ ]:
with open(ROOT / f'similarity_tables/sim-search_{task}_{corpus}_{fuzziness}_{date}.json', "w") as f:
    json.dump(similarity_table, f, ensure_ascii=False)

In [16]:
similarity_table

{'task': 'lieder',
 'date': '25-09-20_12:05',
 'corpus': 'longest',
 'method': 'similarity_search',
 'fuzziness': 80,
 'results': [['E000036',
   1660,
   'Predigt,Paragraph,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung\n0,halleluja gelobet sey gott,18,0,408,r alleluja gelobt fey gott,88.46,False\n1,der wir vergessen sollen zu keiner stunde,18,3,408,der wir vergessen foln zu keiner stunde,95.0,False\n2,groß ist herr deine güte etc,19,10,640,groß ist herr deine treu,84.62,False\n3,a wenn ich nur dich,21,13,768,wann ich nur dich,88.89,False\n4,suche ich errettung aus der noth,21,26,1145,und mich retten aus der noth,80.0,False\n5,alle die zu ihm treten,21,28,590,die zu ihm tre ten,85.0,False\n6,daß mich nicht treffe sein zorngericht,23,10,1000,daß mich nicht treffe dein gericht,91.67,True\n7,jst voller freud und singen,24,1,903,ift voller freud und singen,92.59,False\n8,sieht lauter sonnenschein die sonne die mir lachet,24,2,903,sieht lauter connenschein die sonne,80.0,False\n9,jst mein h

In [15]:
buffer = io.StringIO(similarity_table['results'][0][2])
df2 = pd.read_csv(buffer)
df2

,Predigt,Paragraph,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
0,halleluja gelobet sey gott,18,0,408,r alleluja gelobt fey gott,88.46,False
1,der wir vergessen sollen zu keiner stunde,18,3,408,der wir vergessen foln zu keiner stunde,95.00,False
2,groß ist herr deine güte etc,19,10,640,groß ist herr deine treu,84.62,False
3,a wenn ich nur dich,21,13,768,wann ich nur dich,88.89,False
4,suche ich errettung aus der noth,21,26,1145,und mich retten aus der noth,80.00,False
...,...,...,...,...,...,...,...
181,herr gott dich loben wir,166,6,969,herr gott dich loben wir,100.00,True
182,herr gott dich loben wir,166,6,969,herr gott dich loben wir,100.00,True
183,herr gott dich loben wir,166,6,969,herr gott dich loben wir,100.00,True
184,herr gott dich loben wir,166,6,969,s herr sott dich loben wir,92.00,True


In [18]:
def get_results_csv(list_of_lists, id):
  for sublist in list_of_lists:
    if len(sublist) >= 3 and sublist[0] == id:
      return sublist[2]
  return None

In [ ]:
for id in testsermons:
    # get results for this specific sermon from results tables
    results_csv = get_results_csv(similarity_table['results'], id)
    if results_csv:
        # write results to csv
        filepath = ROOT / f"predictions/{id}_{similarity_table['method']}_{similarity_table['fuzziness']}_{similarity_table['date']}.csv"
        with open(filepath, "w") as f:
            f.write(results_csv)
        
        # collect metadata
        info = {}
        info["date"] = similarity_table['date']
        info["task"] = similarity_table['task']
        info["method"] = similarity_table['method']
        info["fuzziness"] = similarity_table['fuzziness']
        info['file'] = str(filepath)

        # append metadata if file already exists,
        # otherwise create new file
        my_file = Path(ROOT / f"predictions/{id}_predictions.json")
        if my_file.is_file():
            with open(my_file, "a", encoding="utf-8") as f:
                predictions = json.load(f)
                predictions.append(info)
                json.dump(predictions, f, ensure_ascii=False)
        else:
            with open(ROOT / f"predictions/{id}_predictions.json", "x", encoding="utf-8") as f:
                json.dump([info], f, ensure_ascii=False)